### Goal

The objective of this notebook is to collect historical Air Quality Index (AQI) data for [Madison](https://docs.google.com/spreadsheets/d/1pHLA9XzXoy9nJTaiNkgThGPQjVEa0tfeH203I6FA238/edit?gid=0#gid=0&range=E55), located in [Dane County](https://en.wikipedia.org/wiki/Dane_County,_Wisconsin), [Wisconsin](https://docs.google.com/spreadsheets/d/1pHLA9XzXoy9nJTaiNkgThGPQjVEa0tfeH203I6FA238/edit?gid=0#gid=0&range=F55) starting from 1964 to 2024, for dates ranging from May 1st to October 31st.

We utilize the US Environmental Protection Agency (EPA) Air Quality System (AQS) API, which focuses on historical data rather than real-time air quality information. The API [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) provides detailed explanations of the various parameters used in API calls and includes examples of how to retrieve data. For more information about the Air Quality System, refer to the [EPA FAQ](https://www.epa.gov/outdoor-air-quality-data/frequent-questions-about-airdata).

The Air Quality Index measures the air quality on a given day, indicating how safe or polluted the air is. It tracks common pollutants such as smog, smoke and carbon monoxide. A rating between 0 and 50 reflects clean, healthy air, while the index value of 301 or above represents hazardous conditions. A comprehensive explanation of AQI calculation is available [here](https://www.airnow.gov/aqi/aqi-basics/).

The US EPA was created in the early 1970's. The EPA reports that they only started broad based monitoring with standardized quality assurance procedures in the 1980's. Many counties will have data starting somewhere between 1983 and 1988. However, some counties still do not have any air quality monitoring stations. The API helps resolve this by providing calls to search for monitoring stations and data using either station ids, or a county designation or a geographic bounding box.

To locate nearby air quality monitoring stations, Federal Information Processing Series (FIPS) codes are required for the specific city, county, and state. The FIPS data for this notebook was sourced from [federal communications commission](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt). I am using the county level FIPS code for Madison.

State FIPS: 55
County FIPS: 025
County level FIPS: 55025


### License

#### Code Attribution

Snippets of the code were taken from a code example developed by **Dr. David W. McDonald** for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [**Creative Commons CC-BY license**](https://creativecommons.org/licenses/by/4.0/).


#### Step 1: Data Retrieval

Importing essential libraries.

In [1]:
# 
#    These are standard python modules
#
#import json, time, urllib.parse
import json, time
#    The 'requests' module is a distribution module for making web requests. If you do not have it already, you'll need to install it
import requests

import pandas as pd

import warnings

from pyproj import Transformer, Geod

warnings.filterwarnings("ignore")

Defining global constants that will used during API calls and data filtering.

In [2]:
#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are some of the 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
# 
AQS_REQUEST_TEMPLATE = {
    "email":      "manasars@uw.edu",     
    "key":        "",      
    "state":      "55",     # the two digit state FIPS # as a string
    "county":     "025",     # the three digit county FIPS # as a string
    "begin_date": "19640501",     # the start of a time window in YYYYMMDD format
    "end_date":   "20211031",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}

USERNAME = "manasars@uw.edu"
APIKEY = "cobaltgoose48"

STARTYEAR = 1964
ENDYEAR = 2024

Before we can use the API we need to request a key. I am using my email address to make the request. The EPA then sends a confirmation email link and a 'key' that we use for all other requests.
We only need to sign-up once, unless we want to invalidate the current key (by getting a new key) or we lose the key. So this block of code has commented out as the key has been already generated. In case there is a need for generating it again, uncomment the entire cell.

In [3]:
# #
# #    This implements the sign-up request. The parameters are standardized so that this function definition matches
# #    all of the others. However, the easiest way to call this is to simply call this function with your preferred
# #    email address.
# #
# def request_signup(email_address = None,
#                    endpoint_url = API_REQUEST_URL, 
#                    endpoint_action = API_ACTION_SIGNUP, 
#                    request_template = AQS_REQUEST_TEMPLATE,
#                    headers = None):
    
#     # Make sure we have a string - if you don't have access to this email addres, things might go badly for you
#     if email_address:
#         request_template['email'] = email_address        
    
#     if not request_template['email']: 
#         raise Exception("Must supply an email address to call 'request_signup()'")

#     if '@' not in request_template['email']: 
#         raise Exception(f"Must supply an email address to call 'request_signup()'. The string '{request_template['email']}' does not look like an email address.")

#     # Compose the signup url - create a request URL by combining the endpoint_url with the parameters for the request
#     request_url = endpoint_url+endpoint_action.format(**request_template)
        
#     # make the request
#     try:
#         # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
#         # during the request processing - throttling is always a good practice with a free data source
#         if API_THROTTLE_WAIT > 0.0:
#             time.sleep(API_THROTTLE_WAIT)
#         response = requests.get(request_url, headers=headers)
#         json_response = response.json()
#     except Exception as e:
#         print(e)
#         json_response = None
#     return json_response

# print("Requesting SIGNUP ...")
# response = request_signup("manasars@uw.edu")
# print(json.dumps(response,indent=4))

Once we have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. Some of the responses rely on jargon or terms-of-art. So, one needs to know a bit about the way atmospheric sciece works to understand some of the terms.

In [4]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors 
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [5]:
#
#   The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clustors of
#   sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the
#   a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from
#   that specific sensor.
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

We are interested in getting to something that might be the Air Quality Index (AQI). AQI is reported on the news often around smoke and smog values. The AQI is a complex measure of different gasses and of the particles in the air (dust, dirt, ash etc). From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.

In [6]:
#
#   Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group)
#   The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
#   We'll use that to make another list request.
#
AQI_PARAM_CLASS = "AQI POLLUTANTS"


In [7]:
#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We now have a response containing a set of sensor ID numbers. The list includes the sensor numbers as well as a description or name for each sensor.

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up. So we break the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [8]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#   
#

Air quality monitoring stations are located all over the US at different locations. To have AQI data relevant to Madison, we must focus on monitoring stations in and around Madison.
To do so, we must supply the FIPS number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.

In [9]:
#
#   We'll use these the Madison city location
#
CITY_LOCATIONS = {
    'madison' :       {'city'   : 'Madison',
                       'county' : 'Dane',
                       'state'  : 'Wisconsin',
                       'fips'   : '55025',
                       'latlon' : [43.074722,-89.384167] }
}


Given our CITY_LOCATIONS constant we can now find which monitoring locations are nearby. One option is to use the county to define the area we are interest in. You can get the EPA to list their monitoring stations by county. We can also get a set of monitoring stations by using a bounding box of latitude, longitude points. But I am using the county approach.

In [10]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['madison']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['madison']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": null
    },
    {
        "code": "0003",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "0005",
        "value_represented": null
    },
    {
        "code": "0006",
        "value_represented": null
    },
    {
        "code": "0007",
        "value_represented": null
    },
    {
        "code": "0008",
        "value_represented": null
    },
    {
        "code": "0009",
        "value_represented": null
    },
    {
        "code": "0010",
        "value_represented": null
    },
    {
        "code": "0011",
        "value_represented": "LOCATED IN LOT #14, CITY OF FITCHBURG COMMERCIAL PARK"
    },
    {
        "code": "0012",
        "value_represented": null
    },
    {
        "code": "0013",
        "value_represented": null
    },
    {
        "code": "0

The above response gives us a list of monitoring stations. Each monitoring station has a unique "code" which is a string number, and, sometimes, a description. The description seems to be something about where the monitoring station is located. Since we have many monitoring stations in Dane county, we can skip using bounding box approach.

The function below is designed to encapsulate requests to the EPA AQS API. When calling the function we should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call simply pass in the parameters that need to change, like date ranges.

In [11]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date. 
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers
        
    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



The below function takes an existing dataframe and appends new AQI records from a provided dictionary containing AQI data. It extracts relevant fields creating a list of new rows which is then converted into a dataframe. Finally, the function concatenates this new dataframe with the original one and returns the updated dataframe, ensuring that any empty columns are removed in the process. This is to append all our API call responses into one dataframe.

In [12]:
# Function to append the collected AQI data
def concat_aqi_data(df, aqi_data):
    # Create a list to store new rows
    new_rows = []
    
    # Loop through each record in the 'Data' part of the response
    for i in aqi_data['Data']:
        # Create a dictionary for the new row and add it to the list
        new_rows.append({
            'state_code': i['state_code'],
            'county_code': i['county_code'],
            'site_number': i['site_number'],
            'latitude': i['latitude'],
            'longitude': i['longitude'],
            'parameter_code': i['parameter_code'],
            'parameter': i['parameter'],
            'sample_duration': i['sample_duration'],
            'arithmetic_mean': i['arithmetic_mean'],
            'units_of_measure': i['units_of_measure'],
            'date_local': i['date_local'],
            'aqi': i['aqi']
        })
    
    # Convert the list of new rows to a DataFrame and concatenate it with the existing DataFrame
    new_df = pd.DataFrame(new_rows)
    new_df = new_df.dropna(axis=1, how='all')
    df = pd.concat([df, new_df], ignore_index=True)
    
    return df

We now call API to collect AQI data for both gaseous and particulate pollutants over the date range specified in the beginning of this notebook. It initializes an empty DataFrame for each pollutant type and loops through the years to make API calls for daily summaries. If the requests are successful, the corresponding AQI data is appended to the respective DataFrames using the above function. It also prints relevent status information.

In [13]:

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_GASEOUS
request_data['state'] = CITY_LOCATIONS['madison']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['madison']['fips'][2:]

gaseous_responses = []
particulate_responses = []

# Initialize an empty DataFrame for the AQI data
gaseous_aqi_df = pd.DataFrame(columns=['state_code', 'county_code','site_number', 'latitude', 'longitude', 'parameter_code', 
                                       'parameter','sample_duration','arithmetic_mean', 'units_of_measure','date_local', 'aqi'])
particulate_aqi_df = pd.DataFrame(columns=['state_code', 'county_code','site_number', 'latitude', 'longitude', 'parameter_code', 
                                           'parameter','sample_duration','arithmetic_mean', 'units_of_measure', 'date_local', 'aqi'])

# Loop through the years and request data
for year in range(STARTYEAR, ENDYEAR):
    begin_date = f"{year}0501"  # May 1st of the given year
    end_date = f"{year}1031"    # October 31st of the given year

    # Request gaseous data
    request_data['param'] = AQI_PARAMS_GASEOUS
    gaseous_responses = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)
    if gaseous_responses and gaseous_responses["Header"][0]['status'] == "Success":
        print(f"Processing gaseous data for {year}")
        gaseous_aqi_df = concat_aqi_data(gaseous_aqi_df, gaseous_responses)
    else:
        print(f"No gaseous data available for {year}")

    # Request particulate data
    request_data['param'] = AQI_PARAMS_PARTICULATES
    particulate_responses = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)
    if particulate_responses and particulate_responses["Header"][0]['status'] == "Success":
        print(f"Processing particulate data for {year}")
        particulate_aqi_df = concat_aqi_data(particulate_aqi_df, particulate_responses)
    else:
        print(f"No particulate data available for {year}")
    


No gaseous data available for 1964
No particulate data available for 1964
No gaseous data available for 1965
No particulate data available for 1965
No gaseous data available for 1966
No particulate data available for 1966
No gaseous data available for 1967
No particulate data available for 1967
No gaseous data available for 1968
No particulate data available for 1968
No gaseous data available for 1969
No particulate data available for 1969
Processing gaseous data for 1970
No particulate data available for 1970
Processing gaseous data for 1971
No particulate data available for 1971
Processing gaseous data for 1972
No particulate data available for 1972
Processing gaseous data for 1973
No particulate data available for 1973
Processing gaseous data for 1974
No particulate data available for 1974
Processing gaseous data for 1975
No particulate data available for 1975
Processing gaseous data for 1976
No particulate data available for 1976
Processing gaseous data for 1977
No particulate data

We see there is no data available from 1964 - 1969. But we have gaseous data available from 1970 and particles data avalable only from 1989.

Some pollutant parameter data is reported hourly or multiple times per day, potentially leading to duplicate records when values do not change. To prevent this, we will remove duplicates before saving the data to two csv files - [gaseous_aqi_1964_2024.csv](https://github.com/ManasaSRonur/data-512-project/blob/main/intermediary_files/gaseous_aqi_1964_2024.csv), [particulate_aqi_1964_2024.csv](https://github.com/ManasaSRonur/data-512-project/blob/main/intermediary_files/particulate_aqi_1964_2024.csv). These files now have just raw responses from API calls and can be loaded as and when needed for analysis.

In [14]:
gaseous_aqi_df = gaseous_aqi_df.drop_duplicates()
gaseous_aqi_df.to_csv("intermediary_files/gaseous_aqi_1964_2024.csv", index=False)
print("Gaseous AQI Data saved to intermediary_files/gaseous_aqi_1964_2024.csv")

particulate_aqi_df = particulate_aqi_df.drop_duplicates()
particulate_aqi_df.to_csv("intermediary_files/particulate_aqi_1964_2024.csv", index=False)
print("Particulate AQI Data saved to intermediary_files/particulate_aqi_1964_2024.csv")

Gaseous AQI Data saved to intermediary_files/gaseous_aqi_1964_2024.csv
Particulate AQI Data saved to intermediary_files/particulate_aqi_1964_2024.csv


#### Step 2: Data Processing

We will load the previously generated CSV files and combine them into a single dataframe for further analysis.

In [35]:
gaseous_df = pd.read_csv('intermediary_files/gaseous_aqi_1964_2024.csv')
particulate_df = pd.read_csv('intermediary_files/particulate_aqi_1964_2024.csv')

aqi_df = pd.concat([gaseous_df, particulate_df], ignore_index=True)

print("Total records in AQI dataframe is",len(aqi_df))

print("Total Records with blank AQI is",aqi_df['aqi'].isna().sum())


Total records in AQI dataframe is 74718
Total Records with blank AQI is 35625


Nearly half the records do no have AQI data. However we do have gas and particles data available for each record. We will further anlayze if we can use this to calculate AQI values.

In [36]:
aqi_df.value_counts(subset = ['parameter', 'sample_duration'])

parameter                               sample_duration        
Ozone                                   8-HR RUN AVG BEGIN HOUR    18699
                                        1 HOUR                      9436
Sulfur dioxide                          1 HOUR                      7300
                                        3-HR BLK AVG                7287
                                        24-HR BLK AVG               7093
PM2.5 - Local Conditions                1 HOUR                      4279
                                        24-HR BLK AVG               4254
Carbon monoxide                         8-HR RUN AVG END HOUR       3232
                                        1 HOUR                      3230
PM2.5 - Local Conditions                24 HOUR                     2781
Sulfur dioxide                          5 MINUTE                    1655
PM10 Total 0-10um STP                   24 HOUR                     1092
                                        1 HOUR              

The [Technical Assistance Document for the Reporting of Daily Air Quality – the Air Quality Index (AQI)](https://www.airnow.gov/publications/air-quality-index/technical-assistance-document-for-reporting-the-daily-aqi/) in the airnow website has detailed steps to calculate AQI information. A little knowledge from the web on AQI calculation along with this document will serve as base for our AQI calculation.

First step is to define breakpoints with concentration ranges and AQI ranges for each of the 6 pollutants identified above. Breakpoints are in the form of (bp_low, bp_high, i_low, i_high) where bp_low and bp_high are the concentration range in which the pollutant level falls , i_low and i_high are the corresponding AQI range for that concentration level. Next for each record in the dataframe, we retrieve the pollutant concentration value and check if this is in one of the defined breakpoint ranges. If yes we calculate AQI as follows and rounded it to nearest value to match with the exisitng value type.

AQI = (bp_high − bp_low / i_high − i_low) × (concentration − bp_low) + i_low

In [37]:
# Define the breakpoints for each pollutant
breakpoints = {
    'Carbon monoxide': [
        (0, 4.4, 0, 50),
        (4.5, 9.4, 51, 100),
        (9.5, 12.4, 101, 150),
        (12.5, 15.4, 151, 200),
        (15.5, 30.4, 201, 300),
        (30.5, 50.4, 301, 500)
    ],
    'Sulfur dioxide': [
        (0, 35, 0, 50),
        (36, 75, 51, 100),
        (76, 185, 101, 150),
        (186, 304, 151, 200),
        (305, 604, 201, 300),
        (605, 1004, 301, 500)
    ],
    'Nitrogen dioxide (NO2)': [
        (0, 53, 0, 50),
        (54, 100, 51, 100),
        (101, 360, 101, 150),
        (361, 649, 151, 200),
        (650, 1249, 201, 300),
        (1250, 2049, 301, 500)
    ],
    'Ozone': [
        (0, 0.054, 0, 50),
        (0.055, 0.070, 51, 100),
        (0.071, 0.085, 101, 150),
        (0.086, 0.105, 151, 200),
        (0.106, 0.200, 201, 300),
        (0.201, 0.604, 301, 500)
    ],
    'PM10 Total 0-10um STP': [
        (0, 54, 0, 50),
        (54, 154, 51, 100),
        (155, 254, 101, 150),
        (255, 354, 151, 200),
        (355, 424, 201, 300),
        (425, 604, 301, 500)
    ],
    'PM2.5 - Local Conditions': [
        (0, 12.00, 0, 50),
        (12.01, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.4, 301, 500)
    ]
}


def calculate_aqi(row):
    pollutant = row['parameter']
    concentration = round(row['arithmetic_mean'], 2)  # Round to 2 decimal places

    
    # Identify breakpoints for the pollutant
    bp_data = breakpoints.get(pollutant)
    if not bp_data:
        return None  # Skip if no breakpoints defined
    
    # Find the corresponding AQI range
    for (bp_low, bp_high, i_low, i_high) in bp_data:
        if bp_low <= concentration <= bp_high:
            # Calculate AQI using the formula
            aqi = ((i_high - i_low) / (bp_high - bp_low)) * (concentration - bp_low) + i_low
            return round(aqi)
    return None  # Return None if no matching range



We now apply our calculation on the records that have missing AQI values.

In [38]:
# Apply the function only if 'aqi' is blank (NaN), otherwise keep the existing 'aqi' value
aqi_df['aqi'] = aqi_df.apply(lambda row: calculate_aqi(row) if pd.isna(row['aqi']) else row['aqi'], axis=1)

This must have fixed the records with blank AQI values.

In [40]:
print("Total Records with blank AQI is",aqi_df['aqi'].isna().sum())

Total Records with blank AQI is 1705


We still have 1705 records without AQI values, this could be due to concentration value out of defined range. Upon investigation I found these are the records with negative concentration of sulphur dioxde which is clealry a data issue. so we will drop these records.

In [43]:
aqi_df = aqi_df.dropna(subset=['aqi'])

Next we will have to ensure the data we have obtained is from the stations relevant to Madison, so we can reliably use to this to compare smoke data in and around Madison. 

In [44]:
print(f"The stations that returned AQI information are: {aqi_df['site_number'].unique()}")

The stations that returned AQI information are: [  1   2   3   5   8   9 999 898 993 994 998  21  22   7  26  27  31  34
  41  42  47  37  25  48]


Let us analyse the location of these stations, by calculating the proximity of these stations from Madison using the [EPSG:4326](https://www.google.com/url?q=https%3A%2F%2Fepsg.io%2F4326) coordinate system.

In [45]:
site_distance = aqi_df.value_counts(subset=['site_number','latitude', 'longitude']).reset_index()
geodcalc = Geod(ellps='WGS84')
madison = CITY_LOCATIONS['madison']['latlon']
# get distance in miles away from city
site_distance['distance_from_madison'] = site_distance.apply(lambda x: 
    geodcalc.inv(madison[1],madison[0],x['longitude'],x['latitude'])[2]*0.00062137
    , axis=1)
print(site_distance)

    site_number   latitude  longitude  count  distance_from_madison
0            41  43.101010 -89.357680  31145               2.255870
1            27  43.067218 -89.398452   9566               0.889371
2            26  43.117495 -89.362618   8804               3.147467
3            47  43.073300 -89.435800   8754               2.614796
4            22  43.216381 -89.333173   2698              10.112936
5            31  43.091940 -89.359007   2654               1.741661
6            42  43.089996 -89.358729   1818               1.663867
7            34  43.246935 -89.335116   1786              12.143819
8            21  43.090829 -89.357896   1415               1.732996
9             7  43.026941 -89.422620   1268               3.829968
10           25  43.081940 -89.376786    522               0.622710
11           37  43.043054 -89.248727    481               7.195885
12          993  43.114172 -89.360909    439               2.966579
13          898  43.072227 -89.382021    413    

Most stations are within 5 miles of madison except for station 37, 22 and 34 that are 7, 10 and 12 miles away. so the data from all of these stations can be effectively used in our analysis.

#### Step 3: File Generation

We now have clean and processed data, so we will aggregate this data to obtain average AQI for each year bewteen 1st of May to 31st of Oct. This data will be saved as csv, which we will later use for analysis especially to validate our smoke estimation.

In [47]:
# Convert date to datetime and extract year
aqi_df['date_local'] = pd.to_datetime(aqi_df['date_local'])
aqi_df['year'] = aqi_df['date_local'].dt.year

# Calculate the mean AQI per year based on the daily maximum values
average_aqi_per_year = aqi_df.groupby('year')['aqi'].mean().reset_index()

average_aqi_per_year.to_csv('intermediary_files/yearly_aqi_1964_2024.csv', index=False)
